In [30]:
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
from polygon import RESTClient
import dotenv
import os
import pandas as pd
import warnings
import heapq

warnings.filterwarnings('ignore')

dotenv.load_dotenv()
POLYGON_API = os.getenv("POLYGON_API")

client = RESTClient(api_key=POLYGON_API)

In [40]:
'''Taking data from spy stocks to build the strat around'''
#We parse the wikipedia page for spy data

sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-')
tickers = sp500['Symbol'].unique().tolist()

data = yf.download(tickers, period='1y')
data = data.stack()
data.index.names = ['date', 'ticker']
data.columns = data.columns.str.lower()

[*********************100%***********************]  503 of 503 completed


In [82]:
'''Well now filter by dollar volume to only opearte with the stocks which fit our liquidity constraints'''
#We will compute the mean dollar volume from the past year to filter liquid stocks

data['dollarvol'] = (data['close'] * data['volume']) / 1e6
data = data.groupby('ticker').mean() #finding mean dollar vol over past year

data = data.sort_values(by='dollarvol', ascending = False)
data = data.head(100) #top 100 liquid stocks